In [1]:
import os
import torch
import torch.nn as nn
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm

Device Setup

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [14]:
train_dir = '/content/drive/MyDrive/plant_village_updated/plant_village_updated/train'
test_dir = '/content/drive/MyDrive/plant_village_updated/plant_village_updated/test'
val_dir = '/content/drive/MyDrive/plant_village_updated/plant_village_updated/val'

Transforming the data

In [15]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



Loading dataset

In [16]:
train_dataset = datasets.ImageFolder(train_dir,transform=transform)
val_dataset = datasets.ImageFolder(val_dir,transform=transform)
test_dataset = datasets.ImageFolder(test_dir,transform=transform)


Hyperparameters

In [17]:
batch_size = 32
lr=1e-4

Making Dataloader

In [18]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
for train_images,_ in train_dataloader:
    print(train_images.shape)
    print(_)
    break

d:\DiseaseClassification\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


torch.Size([32, 3, 224, 224])
tensor([25, 20,  8,  2,  3, 26, 23, 16,  5, 20, 11, 11,  1,  3, 14, 27, 21, 15,
        28, 15,  1, 14,  6,  6,  9,  0,  9, 14, 12,  1, 22, 10])


Model Initialization

In [20]:
model = models.resnet50(pretrained=True)
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features,num_classes) #to make the model to have the same no of class as i want

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 192MB/s]


Training Pipeline

In [21]:
n_epoch = 10
criterion = nn.CrossEntropyLoss()
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [22]:
checkpoint_dir = './checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)


In [23]:
def save_checkpoint(state, filename):
    torch.save(state, filename)

def load_checkpoint(filepath, model, optimizer):
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Checkpoint loaded: epoch {checkpoint['epoch']}")
    return start_epoch

# Try to load checkpoint if exists (set your checkpoint path)
start_epoch = 1
checkpoint_path = os.path.join(checkpoint_dir, 'best_model.pth')
if os.path.isfile(checkpoint_path):
    start_epoch = load_checkpoint(checkpoint_path, model, optimizer)

In [24]:
def train_one_step(model, criterion, optimizer, images, labels):
    model.train()
    optimizer.zero_grad()

    outputs = model(images)
    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    _, preds = torch.max(outputs, 1)
    correct = (preds == labels).sum().item()
    return loss.item(), correct

def val_one_step(model, criterion, images, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(images)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        correct = (preds == labels).sum().item()
    return loss.item(), correct

In [36]:
patience = 5
best_val_acc = 0
epochs_no_improve = 0

for epoch in range(start_epoch, n_epoch + 1):
    train_epoch_loss = 0
    train_epoch_correct = 0
    total_train = 0

    for images, labels in tqdm(train_dataloader, desc=f'Training Epoch {epoch}/{n_epoch}'):
        images, labels = images.to(device), labels.to(device)
        loss, correct = train_one_step(model, criterion, optimizer, images, labels)
        batch_size = images.size(0)

        train_epoch_loss += loss * batch_size
        train_epoch_correct += correct
        total_train += batch_size

    train_epoch_loss /= total_train
    epoch_train_acc = train_epoch_correct / total_train

    val_epoch_loss = 0
    val_epoch_correct = 0
    total_val = 0

    for images, labels in tqdm(val_dataloader, desc=f'Validation Epoch {epoch}/{n_epoch}'):
        images, labels = images.to(device), labels.to(device)
        loss, correct = val_one_step(model, criterion, images, labels)
        batch_size = images.size(0)

        val_epoch_loss += loss * batch_size
        val_epoch_correct += correct
        total_val += batch_size

    val_epoch_loss /= total_val
    epoch_val_acc = val_epoch_correct / total_val

    scheduler.step()

    print(f"Epoch {epoch}: Train Loss: {train_epoch_loss:.4f}, Train Acc: {epoch_train_acc:.4f}")
    print(f"Epoch {epoch}: Val Loss: {val_epoch_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    # Early Stopping Logic
    if epoch_val_acc > best_val_acc:
        best_val_acc = epoch_val_acc
        epochs_no_improve = 0

        # Save the best model checkpoint
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_epoch_loss,
            'val_loss': val_epoch_loss,
            'train_acc': epoch_train_acc,
            'val_acc': epoch_val_acc,
        }
        best_model_path = os.path.join(checkpoint_dir, 'best_model.pth')
        save_checkpoint(checkpoint, best_model_path)
        print(f"Best model saved at epoch {epoch} with val acc: {best_val_acc:.4f}")

    else:
        epochs_no_improve += 1
        print(f"No improvement in val acc for {epochs_no_improve} epochs.")

    if epochs_no_improve >= patience:
        print(f"Early stopping triggered after {patience} epochs with no improvement.")
        break


Training Epoch 1/10:   0%|          | 0/1678 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


AttributeError: 'float' object has no attribute 'item'

## Training Curve

> Add blockquote



In [33]:
import torch
from torchvision import models
import torch.nn as nn

# Recreate the same model architecture
model = models.resnet50(pretrained=False)  # No need for pretrained now
num_classes = len(train_dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Load the best checkpoint
checkpoint = torch.load("/content/checkpoints/best_model.pth", map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model = model.to(device)
model.eval()

print(f"Loaded model from epoch {checkpoint['epoch']} with val_acc={checkpoint['val_acc']:.4f}")


Loaded model from epoch 10 with val_acc=0.9993


In [35]:
import matplotlib.pyplot as plt

# Accuracy
plt.figure(figsize=(8,5))
plt.plot(train_acc, 'r.-', label='Train Accuracy')
plt.plot(val_acc, 'g.-', label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# Loss
plt.figure(figsize=(8,5))
plt.plot(train_loss, 'r.-', label='Train Loss')
plt.plot(val_loss, 'g.-', label='Validation Loss')
plt.xlabel("Epoch")
plt.ylabel("Cross Entropy Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'train_acc' is not defined

<Figure size 800x500 with 0 Axes>

NameError: name 'evaluate' is not defined

In [31]:
# Accuracy
plt.figure(figsize=(8,5))
plt.plot(train_acc, 'r.-', label='Train Accuracy')
plt.plot(val_acc, 'g.-', label='Validation Accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'train_acc' is not defined

<Figure size 800x500 with 0 Axes>

In [ ]:
plt.figure()
plt.plot(train_loss, 'r.-')
plt.plot(val_loss, 'g.-')
plt.xlabel("Epoch")
plt.ylabel('Cross Entropy Loss')
plt.title("Training Loss")
plt.legend(['Training Loss', 'Validation Loss'])
plt.show()

## Evaluation


In [ ]:
import torch.nn.functional as F

@torch.no_grad()
def evaluate(model, test_dataloader):
    model.eval()
    total_correct = 0
    total_samples = 0

    for test_images, test_labels in tqdm(test_dataloader, desc="Evaluating"):
        test_images = test_images.to(device)
        test_labels = test_labels.to(device)

        outputs = model(test_images)  # shape: (batch_size, num_classes)
        _, preds = torch.max(outputs, 1)  # predicted class indices
        total_correct += (preds == test_labels).sum().item()
        total_samples += test_labels.size(0)

    accuracy = total_correct / total_samples
    return accuracy

print("Accuracy of model on test dataset: ", evaluate(model, test_dataloader))
